In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from simpletransformers.ner import NERModel, NERArgs


In [ ]:
data = pd.read_csv("Formatted.csv")
dev_data = pd.read_csv("Formatted_dev.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder
data['sentence_id'] = LabelEncoder().fit_transform(data['sentence_id'])
dev_data['sentence_id'] = LabelEncoder().fit_transform(dev_data['sentence_id'])

In [ ]:
data.rename(columns = {'word':'words', 'tag':'labels'}, inplace=True)
dev_data.rename(columns = {'word':'words', 'tag':'labels'}, inplace=True)

In [ ]:
data['word'] = data['word'].str.lower()
dev_data['word'] = dev_data['word'].str.lower()

In [ ]:
train_data = {'sentence_id':[], 'words':[],'labels':[]}
test_data = {'sentence_id':[], 'words':[],'labels':[]}
train_data = data[['sentence_id', 'words','labels','word_id']]
test_data = dev_data[['sentence_id', 'words', 'labels', 'word_id']]

In [ ]:
label = data['labels'].unique().tolist()

In [ ]:
args = NERArgs()
args.num_train_epochs = 5
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size=32
args.eval_batch_size=32

In [ ]:
ner_model = NERModel('roberta', 'roberta-base', labels=label, args = args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

In [ ]:
ner_model.train_model(train_data, acc = accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/189 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 5:   0%|          | 0/189 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/189 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/189 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/189 [00:00<?, ?it/s]

(945, 0.2985297672292858)

In [ ]:
result, model_outputs, preds_list = ner_model.eval_model(train_data, output_dir = 'eval/')
print('Training result ', result)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/189 [00:00<?, ?it/s]

Training result  {'eval_loss': 0.06746457303287806, 'precision': 0.8367810072444793, 'recall': 0.8635381012430193, 'f1_score': 0.8499490225630569}


In [ ]:
result, model_outputs, test_preds_list = ner_model.eval_model(test_data, output_dir = 'eval/')
print('Testing result ',result)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/24 [00:00<?, ?it/s]

Testing result  {'eval_loss': 0.652771374831597, 'precision': 0.5522486772486772, 'recall': 0.5938833570412517, 'f1_score': 0.5723098012337218}


In [ ]:
def getSpanRange(pred, df):
  res = []
  row = 0
  for i in range(len(pred)):
    sn = len(pred[i])
    start = 0
    In = False
    spans = []
    for j in range(sn):
      if (not In and pred[i][j] == 'B'):
        start = df.iloc[row]['word_id']
        In = True
      elif(In and (pred[i][j] == 'O' or pred[i][j] == 'B') and df.iloc[row]['word_id']!=start):
        end = df.iloc[row-1]['word_id']
        spans.append((start, end))
        In = False
        if(pred[i][j] == 'B'):
          In = True
          start = df.iloc[row]['word_id']
      row+=1
    if(In):
      end = df.iloc[row-1]['word_id']
      spans.append((start, end))
      In = False
    res.append(spans)
  correct = []
  sid = ''
  temp = []
  start = 0
  end = 0
  In = False
  for row in range(len(df. index)):
    if sid != df.iloc[row]['sentence_id']:
      sid = df.iloc[row]['sentence_id']
      end = df.iloc[row-1]['word_id']
      if(In):
        temp.append((start, end))
        In = False
      correct.append(temp)
      temp = []
    if(not In and df.iloc[row]['labels'] == 'B'):
      start = df.iloc[row]['word_id']
      In = True
    elif(In and (df.iloc[row]['labels'] == 'O' or df.iloc[row]['labels'] == 'B') and df.iloc[row-1]['word_id']!=start):
      end = df.iloc[row-1]['word_id']
      temp.append((start, end))
      In = False
      if(df.iloc[row]['labels'] == 'B'):
          In = True
          start = df.iloc[row]['word_id']
  del correct[0]
    
      
    
  return res, correct
  

In [ ]:
span = getSpanRange(test_preds_list, test_data)

In [ ]:
for i in range(0,30):
  print(span[0][i], span[1][i])

[(0, 8), (11, 23)] [(11, 23)]
[(15, 25), (26, 26)] [(15, 25)]
[(2, 2), (3, 10)] [(3, 10)]
[(1, 12)] [(1, 12)]
[(7, 11), (12, 20)] [(7, 9), (10, 20)]
[(0, 7)] [(0, 7)]
[] [(3, 9)]
[(0, 16)] [(0, 16)]
[] [(7, 11)]
[(2, 16)] [(2, 16)]
[(14, 17), (21, 27)] [(2, 13)]
[(0, 5)] [(0, 5)]
[(0, 15), (17, 23), (24, 28)] [(17, 22), (24, 28)]
[(0, 25)] [(0, 6)]
[(4, 8)] [(4, 8)]
[(17, 20)] [(17, 20)]
[(0, 11)] [(0, 11)]
[(2, 13), (26, 30)] [(2, 13), (26, 30)]
[(0, 13), (29, 41)] [(0, 13), (29, 41)]
[(2, 10)] [(2, 10)]
[(0, 9)] [(0, 9)]
[(0, 4), (5, 10)] [(0, 4), (5, 10)]
[(2, 8)] [(2, 8)]
[(0, 4), (16, 26)] [(0, 4), (16, 26)]
[(0, 5)] [(0, 7)]
[(2, 8)] [(2, 8), (10, 16)]
[(0, 24)] [(0, 24)]
[(1, 6)] [(1, 6)]
[(8, 14)] [(8, 14)]
[(3, 8)] [(4, 8)]


In [ ]:
result

{'eval_loss': 0.4238509349524975,
 'precision': 0.4927360774818402,
 'recall': 0.5789473684210527,
 'f1_score': 0.5323741007194245}

In [ ]:
input = ['username', ' username', ' Without', ' hope', ', we', ' have', ' nothing', '!  The', ' only', ' real', ' hope', ' of', ' ending', ' the', ' epidemic', ' is', ' a', ' cure', '; especially', ' for', ' those', ' who', ' have', ' stage', '3', '-HIV', '. ðŸ’”\n#CommitToACure']
s = ''.join(input)
prediction, model_output = ner_model.predict([s])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'username': 'O'},
  {'username': 'O'},
  {'Without': 'O'},
  {'hope,': 'O'},
  {'we': 'O'},
  {'have': 'O'},
  {'nothing!': 'O'},
  {'The': 'B'},
  {'only': 'I'},
  {'real': 'I'},
  {'hope': 'I'},
  {'of': 'I'},
  {'ending': 'I'},
  {'the': 'I'},
  {'epidemic': 'I'},
  {'is': 'I'},
  {'a': 'I'},
  {'cure;': 'I'},
  {'especially': 'I'},
  {'for': 'I'},
  {'those': 'I'},
  {'who': 'I'},
  {'have': 'I'},
  {'stage3-HIV.': 'I'},
  {'ðŸ’”': 'O'},
  {'#CommitToACure': 'O'}]]